In [ ]:
# GOES Lightning Detector and IR Plots

# Author: Justin Richling - The COMET Program



# Thanks to deeplycloudy for the ```glmtools``` package
# * https://github.com/deeplycloudy/glmtools


# Imports

import os,glob

import numpy as np

from datetime import datetime, timedelta

import matplotlib
from matplotlib.cm import get_cmap
from matplotlib.colors import LogNorm, Normalize
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl
from matplotlib import patheffects
from matplotlib.ticker import LogFormatter 

from netCDF4 import Dataset

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

import xarray as xr

from siphon.cdmr import Dataset as siphonDataset
from siphon.radarserver import RadarServer
from metpy.plots import ctables  # For NWS colortable
#reflec_norm, reflec_cmap = ctables.registry.get_with_steps('NWSReflectivity', 5, 5)
reflec_norm, reflec_cmap = ctables.registry.get_with_steps('NWSStormClearReflectivity',-20., 0.5)
#vel_norm, vel_cmap = ctables.registry.get_with_range('NWSVelocity', -120, 105)
vel_norm, vel_cmap = ctables.registry.get_with_range('NWSVelocity', -100, 100)


def velocity_cmap():
    import pandas as pd
    rad_vel = pd.read_csv("/Users/chowdahead/Downloads/radial_velocity_cmap.csv")
    rad_vel_colors = []
    for i in range(0,256):
        rad_vel_colors.append((float(rad_vel["r"][i]),
                               float(rad_vel["g"][i]),
                               float(rad_vel["b"][i]),
                               float(rad_vel["a"][i])
                             ))
    cmap_name="rad_vel"
    rad_vel_cmap = LinearSegmentedColormap.from_list(
            cmap_name, rad_vel_colors)
    return rad_vel_cmap

# Plot Functions

plt.rcParams.keys()

# Radar

def datestdtojd (stddate):
    fmt='%Y-%m-%d'
    sdtdate = datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = "0"+str(sdtdate.tm_yday)
    return(jdate)
#datestdtojd("2018-01-10")

def raw_to_masked_float(var, data):
    # Values come back signed. If the _Unsigned attribute is set, we need to convert
    # from the range [-127, 128] to [0, 255].
    if var._Unsigned:
        data = data & 255

    # Mask missing points
    data = np.ma.array(data, mask=data==0)

    # Convert to float using the scale and offset
    return data * var.scale_factor + var.add_offset

def polar_to_cartesian(az, rng):
    az_rad = np.deg2rad(az)[:, None]
    x = rng * np.sin(az_rad)
    y = rng * np.cos(az_rad)
    return x, y

def Radar_Map(Year,Month,Day,Hour,Minute,station,Extent,prod,im_path,radar_only=True):
#def Radar_Map(Year,Month,Day,Hour,Minute,station,SAT_samples,index,Extent,sat,prod,case,radar_only,
#             im_path):
    """
    FigSize must be used as a tuple:
    (12,12) or (17,10), etc.
    
    """
    
    import pandas as pd
    rad_vel = pd.read_csv("/Users/chowdahead/Downloads/radial_velocity_cmap.csv")
    rad_vel_colors = []
    for i in range(0,256):
        rad_vel_colors.append((float(rad_vel["r"][i]),
                               float(rad_vel["g"][i]),
                               float(rad_vel["b"][i]),
                               float(rad_vel["a"][i])
                             ))
    cmap_name="rad_vel"
    rad_vel_cmap = LinearSegmentedColormap.from_list(
            cmap_name, rad_vel_colors)
    
    rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
    query = rs.query()
    now = datetime(Year,Month,Day,Hour,Minute)
    query.stations(station).time(now) 

    rs.validate_query(query)
    catalog = rs.get_catalog(query)
    file_station = str(catalog.datasets[0])
    file_station = file_station[0:4]
    #print(file_station)
    ds = list(catalog.datasets.values())[0]
    data = siphonDataset(ds.access_urls['CdmRemote'])
    
    time_stamp = data.time_coverage_start[:10].replace("-","_")+\
        "_"+data.time_coverage_start[11:16].replace(":","")
    print("radar time:",time_stamp)
    sweep = 0
    if prod == "vel":
        ref_var = data.variables['RadialVelocity_HI']
        ref_data = ref_var[sweep]
        rng = data.variables['distanceV_HI'][:]
        az = data.variables['azimuthV_HI'][sweep]
        
    if prod == "refl":
        ref_var = data.variables['Reflectivity_HI']
        ref_data = ref_var[sweep]
        rng = data.variables['distanceR_HI'][:]
        az = data.variables['azimuthR_HI'][sweep]
        
    ref = raw_to_masked_float(ref_var, ref_data)
    x, y = polar_to_cartesian(az, rng)
    
    '''
    sat_ds = Dataset(SAT_samples[index])
    # Extract the IR values from the NetCDF
    sat_data = sat_ds.variables['CMI'][:]
    Time = str(datetime.strptime(sat_ds.time_coverage_start[:-6], '%Y-%m-%dT%H:%M'))
    Time = Time[0:10]+Time[11:16]
    timestamp = datetime.strptime(Time, '%Y-%m-%d%H:%M')
    file_time = timestamp.strftime('%Y_%m_%d_%H%M')
    print("sat time:",file_time,"\n")
    
    # Coordinate values for plotting
    #-----------------------------------------------------------
    proj_var = sat_ds.variables['goes_imager_projection']
    sat_x = sat_ds.variables['x'] * proj_var.perspective_point_height
    sat_y = sat_ds.variables['y'] * proj_var.perspective_point_height
    '''
    fig = plt.figure(figsize=(10, 10))

    #globe = ccrs.Globe(semimajor_axis=proj_var.semi_major_axis, 
    #                   semiminor_axis=proj_var.semi_minor_axis)
    
    #proj = ccrs.Geostationary(central_longitude=proj_var.longitude_of_projection_origin,
    #                   satellite_height=proj_var.perspective_point_height, globe=globe)
    
    # Create projection centered on the radar. This allows us to use x
    # and y relative to the radar.
    radar_proj = cartopy.crs.LambertConformal(central_longitude=data.StationLongitude, 
                                              central_latitude=data.StationLatitude)
    
    merc_globe = ccrs.Globe(semimajor_axis=6371229, 
                            semiminor_axis=6371229,
                           datum='WGS84')
    
    ax_proj = ccrs.Mercator(central_longitude=0.0,
        #min_latitude=34.5,
        #max_latitude=40.7,
        globe=merc_globe,
        false_easting=0.0,
        false_northing=0.0,
        scale_factor=1.0)
    
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal())
    
    state_boundaries = cfeature.NaturalEarthFeature(category='cultural',
                            name='admin_1_states_provinces_lakes',
                            scale='10m', facecolor='none',
                            linestyle=":",linewidth=1)
    ax.coastlines('10m', color='r',linewidth=1)
    ax.add_feature(state_boundaries, edgecolor='r')

    #reader = shpreader.Reader('/Users/chowdahead/Downloads/countyl010g_shp_nt00964/countyl010g.shp')
    #counties = list(reader.geometries())
    #COUNTIES = cartopy.feature.ShapelyFeature(counties, cartopy.crs.PlateCarree())
    #ax.add_feature(COUNTIES, facecolor='none', edgecolor='red',alpha=0.5)
    #ax.set_extent(Extent,ccrs.PlateCarree())
    
    if radar_only == True:
        file_ext = prod
  
    if radar_only == False:
        if sat == "ir":
            # Plot IR
            #----------------------------------------------------------
            sat_img = ax.imshow(sat_data, extent=(sat_x.min(), sat_x.max(), 
                              sat_y.min(), sat_y.max()), origin='upper',
                              interpolation='nearest',
                              cmap="Greys",transform=proj)

        if sat == "vis":
            # Plot VIS
            #----------------------------------------------------------
            sat_img = ax.imshow(np.sqrt(sat_data), extent=(sat_x.min(), sat_x.max(), 
                              sat_y.min(), sat_y.max()), origin='upper',
                              interpolation='nearest',
                              cmap="Greys_r",transform=proj)
        
        file_ext = f'{prod}_{sat}'
    print(ref.min(),ref.max())
    if prod == "vel":
        ticks = [-100,-80,-60,-45,-20,0,20,45,60,80,100]
        rad_cmap = rad_vel_cmap
        rad_norm = vel_norm
        cs = ax.pcolormesh(x, y, ref, cmap=rad_cmap, zorder=100,transform=radar_proj,
                          vmin=-80,vmax=80)#norm= rad_norm
        
    if prod == "refl":
        ticks = np.arange(-10,80,10)
        rad_cmap = reflec_cmap
        rad_norm = reflec_norm
        cs = ax.pcolormesh(x, y, ref, cmap=rad_cmap, norm=rad_norm, zorder=100,transform=radar_proj)
    
    cbar = plt.colorbar(cs, orientation='horizontal')
    posn = ax.get_position()
    outline_effect = [patheffects.withStroke(linewidth=3, foreground='k')]
    cbar.ax.set_position([posn.x0+0.001, posn.y0-0.001,
                            (posn.x1-posn.x0)/2, posn.height])
    params = {
              "xtick.color" : "k",
              "ytick.color" : "k",
              "font.size" : 10,
                  }
    #plt.rcParams.update(params)
    cbar.set_ticks([])
    cbar.ax.set_xticklabels([])
    
    if prod == "vel":
        Y = -11
        cbar.ax.text(-76, Y, "RF", ha='center', va='center',path_effects=outline_effect,color="w") #RF
        #cbar.ax.text(-100, Y, ticks[0], ha='center', va='center',path_effects=outline_effect,color="w") #-80
        #cbar.ax.text(-80, Y, ticks[1], ha='center', va='center',path_effects=outline_effect,color="w") #-60
        cbar.ax.text(-60, Y, ticks[2], ha='center', va='center',path_effects=outline_effect,color="w") #-20
        cbar.ax.text(-45, Y, ticks[3], ha='center', va='center',path_effects=outline_effect,color="w")#20
        cbar.ax.text(-20, Y, ticks[4], ha='center', va='center',path_effects=outline_effect,color="w")#45
        cbar.ax.text(0, Y, ticks[5], ha='center', va='center',path_effects=outline_effect,color="w")#80
        cbar.ax.text(20, Y, ticks[6], ha='center', va='center',path_effects=outline_effect,color="w")#20
        cbar.ax.text(45, Y, ticks[7], ha='center', va='center',path_effects=outline_effect,color="w")#45
        cbar.ax.text(60, Y, ticks[8], ha='center', va='center',path_effects=outline_effect,color="w")#80
        #cbar.ax.text(80, Y, ticks[9], ha='center', va='center',path_effects=outline_effect,color="w")#45
        #cbar.ax.text(100, Y, ticks[10], ha='center', va='center',path_effects=outline_effect,color="w")#80

    if prod == "refl":
        Y = 23
        cbar.ax.text(-10, Y, ticks[0], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(0, Y, ticks[1], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(10, Y, ticks[2], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(20, Y, ticks[3], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(30, Y, ticks[4], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(40, Y, ticks[5], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(50, Y, ticks[6], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(60, Y, ticks[7], ha='center', va='center',path_effects=outline_effect,color="w")
        cbar.ax.text(70, Y, ticks[8], ha='center', va='center',path_effects=outline_effect,color="w")
    
    
    prod_time = ax.text(0.99, 0.01, time_stamp,
                        horizontalalignment='right', transform=ax.transAxes,
                        color='white', fontsize=7, weight='bold')
    outline_effect = [patheffects.withStroke(linewidth=2.5, foreground='black')]
    prod_time.set_path_effects(outline_effect)
    
    plt.savefig(f"{im_path}{time_stamp}_{file_ext}_{station}.png",
                bbox_inches="tight",dpi=100)
    plt.close(fig)

# Make lists for Radar times that correspond to Sat times

# function to get unique values 
def unique(list1): 
      
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    return unique_list

def get_radar_sat_times(year,month,day,start,end,station):
    radar_times = []
    #day = 10
    for j in range(start,end):
        for i in range(0,60):
            rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
            query = rs.query()
            now = datetime(year, month, int(day), j,i)
            
            query.stations(station).time(now) 
            #query.stations(station).time_range(start, end)

            rs.validate_query(query)
            catalog = rs.get_catalog(query)

            ds = list(catalog.datasets.values())[0]
            data = siphonDataset(ds.access_urls['CdmRemote'])

            radar_times.append(data.time_coverage_start[:4]+datestdtojd(f"2018-01-{day}")
                  +data.time_coverage_start[11:16].replace(":",""))
    print("done.")
    return radar_times

def get_radar_lists(day, start, end, stations_list, VIS_list):
    
    radar_times = get_radar_sat_times(day,start,end,stations_list)

    radar_times_fin = unique(radar_times)
    radar_times = list(np.sort(radar_times_fin))

    sat_index = []
    for i in VIS_list[:]:
        for ass in radar_times:
            if ass == i[-49:-38]:
                sat_index.append(VIS_list.index(i))
                
    return sat_index,radar_times

# Write the Radar times to list and save as a file just in case...

def make_text_file(case_path,station,sat_index,radar_times_fin):
    with open(f"{case_path}radar_time_sat_index_{station}.txt", "w") as f:
        for s in sat_index:
            f.write(str(s) +"\n")

    with open(f"{case_path}radar_time_index_{station}.txt", "w") as f:
        for s in radar_times_fin:
            f.write(str(s) +"\n")
    print(f"{case_path}radar_time_sat_index_{station}.txt")
    print(f"{case_path}radar_time_index_{station}.txt")
    return

# Try to read in the saved satellite indices

def read_text_file(case_path,station,sat_index,radar_times_fin):
    with open(f"{tx_path}radar_time_sat_index.txt", "r") as f:
      for line in f:
        radar_index_saved.append(int(line.strip()))

    with open(f"{tx_path}radar_time_sat_index.txt", "r") as f:
      for line in f:
        sat_index_saved.append(int(line.strip()))
    return sat_index_saved